# Preprocessing Nebraska Reservoir and Observation Site data for WaDE

In [1]:
#Needed Libararies

# working with data
import os
import numpy as np
import pandas as pd
import geopandas as gpd

# visulizaiton
import matplotlib.pyplot as plt
import seaborn as sns

# API retrieval
import requests
import json

# Cleanup
from datetime import datetime
pd.set_option('display.max_columns', 999)  # How to display all columns of a Pandas DataFrame in Jupyter Notebook

In [2]:
# Working Directory
workingDir = "G:/Shared drives/WaDE Data/Nebraska/SS_ReservoirsObservationSites/RawInputData"
os.chdir(workingDir)

## API Site Data

In [3]:
# done already

# %%time
# # Retrieve list of only NeDNRstream gage stations
# #######################################

# # API retrieval
# url = "https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/GetStationList"
# responseD = json.loads(requests.get(url).text)
# DtL = responseD['Results']
# length = len(DtL)

# # create dataframe and store
# df = pd.DataFrame()
# for i in range(length):
#     row = pd.DataFrame([DtL[i]])
#     df = pd.concat([df, row])

# # Use only NeNDR Active provided sites
# df = df[df['SourceName'] == 'NeDNR']

# # Exporting output files.
# df.to_csv('StreamGageGetStationList.csv', index=False)  # The output.

In [4]:
# the one we want to test out

# Input File - StreamGageGetStationList.csv
sgInput = "StreamGageGetStationList.csv"
dfsg = pd.read_csv(sgInput)

# WaDE UUID tracker for data assessment
if 'WaDEUUID' not in dfsg:
    dfsg['WaDEUUID'] = "nebRG" + dfsg.index.astype(str)
    dfsg.to_csv('StreamGageGetStationList.csv', index=False)

dfsg.head(1)

,StationName,SourceName,StationNumber,RiverBasin,FieldOffice,StationTypeDescription,Downstream,TimeZone,Elevation,ElevationUnits,Longitude,Latitude,IsActive,WaDEUUID
0,Parks Canal from Republican River,NeDNR,120500,Republican River,NaN,Canal/Pump,0,MST,0.0,ft,-101.733703,40.030071,True,nebRG0


In [5]:
# Update StationNumber values must have 8 digits, with leading 0s

def updateStationNumber(x):
    x = str(x).strip()
    if len(x) == 4:
        x = "0000" + x
    if len(x) == 5:
        x = "000" + x
    if len(x) == 6:
        x = "00" + x
    if len(x) == 7:
        x = "0" + x
    return x

dfsg['StationNumber'] = dfsg.apply(lambda row: updateStationNumber(row['StationNumber']), axis=1)
dfsg.head()

,StationName,SourceName,StationNumber,RiverBasin,FieldOffice,StationTypeDescription,Downstream,TimeZone,Elevation,ElevationUnits,Longitude,Latitude,IsActive,WaDEUUID
0,Parks Canal from Republican River,NeDNR,00120500,Republican River,NaN,Canal/Pump,0,MST,0.000,ft,-101.733703,40.030071,True,nebRG0
1,Hat Creek at Montrose,NeDNR,00062500,White River,Bridgeport,Stream,1500,MST,3635.126,ft,-103.742800,42.922260,True,nebRG1
2,White River at Crawford,NeDNR,06444000,White River,Bridgeport,Stream,2300,MST,3658.297,ft,-103.417700,42.686640,True,nebRG2
3,Whitney Reservoir from White River,NeDNR,00410000,White River,Bridgeport,Reservoir,2350,MST,0.000,ft,-103.309400,42.792110,True,nebRG3
4,Niobrara River at Wyoming-Nebraska State,NeDNR,06454000,Niobrara River,Bridgeport,Stream,4005,MST,4685.993,ft,-104.051600,42.652520,True,nebRG4


In [6]:
# Get list of StationNumber
streamgageIdList = dfsg['StationNumber'].tolist()   
print(len(streamgageIdList))
streamgageIdList

210


['00120500',
 '00062500',
 '06444000',
 '00410000',
 '06454000',
 '00072000',
 '00079000',
 '06454050',
 '00038100',
 '00038200',
 '00084000',
 '00029000',
 '00062000',
 '06454100',
 '00078000',
 '00089000',
 '00013000',
 '00104000',
 '00063000',
 '00086000',
 '00046000',
 '00069000',
 '00123000',
 '06454500',
 '06455500',
 '00081000',
 '00100000',
 '06456500',
 '06457500',
 '06459025',
 '06459175',
 '00001000',
 '06459500',
 '06464900',
 '06464930',
 '06465700',
 '00225000',
 '06678500',
 '00101100',
 '00056000',
 '00145100',
 '00002692',
 '06677500',
 '06678000',
 '00040000',
 '06679000',
 '06679500',
 '06680000',
 '00148000',
 '00149000',
 '00022000',
 '06681000',
 '00099000',
 '06680970',
 '00021000',
 '00144400',
 '06682000',
 '06681999',
 '06681998',
 '00106000',
 '00106100',
 '06682500',
 '00133000',
 '00024000',
 '00002000',
 '00003000',
 '00115100',
 '00002696',
 '06683000',
 '06684000',
 '00009000',
 '06684500',
 '06684498',
 '06684499',
 '06685000',
 '00010000',
 '00039000',

## Timeseries Data

### DailyMeanByYear

In [7]:
# get timeseires without using the year list

# %%time
# # get timeseries results
# # use StationNumber in url

# # create empty dataframe
# dfts = pd.DataFrame()

# sglength = len(streamgageIdList)
# for i in range(sglength):
#     serviceStr = "DailyMeanByYear" # change here
#     url = "https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/" + serviceStr + "?StationNumber=" + str(streamgageIdList[i])
#     try:
#         responseD = json.loads(requests.get(url).text)
#         DtL = responseD['Results']

#         # store in dataframe
#         dftemp = pd.DataFrame()
#         length = len(DtL)
#         for x in range(length):
#             row = pd.DataFrame([DtL[x]])
#             row['url'] = url
#             row['service'] = serviceStr
#             dftemp = pd.concat([dftemp, row])

#         dfts = pd.concat([dfts, dftemp])
    
#     except:
#         dftemp = pd.DataFrame()
#         dftemp['url'] = url
#         dfts = pd.concat([dfts, dftemp])
#         print("Error, issue with API return.")

# dfts.to_csv('DailyMeanByYear.csv', index=False)  # The output.
# print(len(dfts))
# dfts.head()

In [8]:
# Year list
yearList = ["2000", "2001", "2002", "2003", "2004", "2005", "2006", "2007", "2008", "2009",
            "2010", "2011", "2012", "2013", "2014", "2015", "2016", "2017", "2018", "2019",
            "2020", "2021", "2022"]

In [9]:
%%time
# get timeseries results
# use StationNumber in url

# create empty dataframe
dfts = pd.DataFrame()

sgLength = len(streamgageIdList)
ylLength = len(yearList)

for i in range(sgLength):
    serviceStr = "DailyMeanByYear" # change here
    for j in range(ylLength):
        url = "https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/" + serviceStr + "?StationNumber=" + str(streamgageIdList[i]) + "&MeanYear=" + str(yearList[j])
        print(url)
        try:
            responseD = json.loads(requests.get(url).text)
            DtL = responseD['Results']

            # store in dataframe
            dftemp = pd.DataFrame()
            length = len(DtL)
            for x in range(length):
                row = pd.DataFrame([DtL[x]])
                row['url'] = url
                row['service'] = serviceStr
                dftemp = pd.concat([dftemp, row])

            dfts = pd.concat([dfts, dftemp])
        except:
            dftemp = pd.DataFrame()
            dftemp['url'] = url
            dfts = pd.concat([dfts, dftemp])
            print("Error, issue with API return.")

print(len(dfts))
dfts.head()

https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=00120500&MeanYear=2000
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=00120500&MeanYear=2001
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=00120500&MeanYear=2002
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=00120500&MeanYear=2003
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=00120500&MeanYear=2004
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=00120500&MeanYear=2005
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=00120500&MeanYear=2006
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=00120500&MeanYear=2007
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=00120500&MeanYear=2008
https://nednr.nebraska.gov/IwipApi/api/v1/Stre

https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=00410000&MeanYear=2009
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=00410000&MeanYear=2010
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=00410000&MeanYear=2011
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=00410000&MeanYear=2012
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=00410000&MeanYear=2013
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=00410000&MeanYear=2014
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=00410000&MeanYear=2015
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=00410000&MeanYear=2016
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=00410000&MeanYear=2017
https://nednr.nebraska.gov/IwipApi/api/v1/Stre

https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=00079000&MeanYear=2018
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=00079000&MeanYear=2019
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=00079000&MeanYear=2020
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=00079000&MeanYear=2021
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=00079000&MeanYear=2022
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=06454050&MeanYear=2000
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=06454050&MeanYear=2001
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=06454050&MeanYear=2002
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=06454050&MeanYear=2003
https://nednr.nebraska.gov/IwipApi/api/v1/Stre

https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=00084000&MeanYear=2004
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=00084000&MeanYear=2005
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=00084000&MeanYear=2006
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=00084000&MeanYear=2007
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=00084000&MeanYear=2008
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=00084000&MeanYear=2009
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=00084000&MeanYear=2010
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=00084000&MeanYear=2011
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=00084000&MeanYear=2012
https://nednr.nebraska.gov/IwipApi/api/v1/Stre

https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=06454100&MeanYear=2013
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=06454100&MeanYear=2014
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=06454100&MeanYear=2015
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=06454100&MeanYear=2016
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=06454100&MeanYear=2017
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=06454100&MeanYear=2018
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=06454100&MeanYear=2019
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=06454100&MeanYear=2020
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=06454100&MeanYear=2021
https://nednr.nebraska.gov/IwipApi/api/v1/Stre

...

https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=06844000&MeanYear=2016
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=06844000&MeanYear=2017
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=06844000&MeanYear=2018
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=06844000&MeanYear=2019
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=06844000&MeanYear=2020
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=06844000&MeanYear=2021
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=06844000&MeanYear=2022
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=00231700&MeanYear=2000
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=00231700&MeanYear=2001
https://nednr.nebraska.gov/IwipApi/api/v1/Stre

https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=06851090&MeanYear=2002
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=06851090&MeanYear=2003
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=06851090&MeanYear=2004
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=06851090&MeanYear=2005
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=06851090&MeanYear=2006
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=06851090&MeanYear=2007
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=06851090&MeanYear=2008
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=06851090&MeanYear=2009
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=06851090&MeanYear=2010
https://nednr.nebraska.gov/IwipApi/api/v1/Stre

https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=06880000&MeanYear=2011
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=06880000&MeanYear=2012
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=06880000&MeanYear=2013
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=06880000&MeanYear=2014
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=06880000&MeanYear=2015
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=06880000&MeanYear=2016
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=06880000&MeanYear=2017
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=06880000&MeanYear=2018
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=06880000&MeanYear=2019
https://nednr.nebraska.gov/IwipApi/api/v1/Stre

https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=06881500&MeanYear=2020
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=06881500&MeanYear=2021
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=06881500&MeanYear=2022
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=06883555&MeanYear=2000
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=06883555&MeanYear=2001
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=06883555&MeanYear=2002
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=06883555&MeanYear=2003
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=06883555&MeanYear=2004
https://nednr.nebraska.gov/IwipApi/api/v1/StreamGage/DailyMeanByYear?StationNumber=06883555&MeanYear=2005
https://nednr.nebraska.gov/IwipApi/api/v1/Stre

,StationNumber,Date,Value,Status,Qualifier,Unit,url,service
0,00120500,2008-10-01T00:00:00-05:00,8.8,Approved,,ft^3/s,https://nednr.nebraska.gov/IwipApi/api/v1/Stre...,DailyMeanByYear
0,00120500,2008-10-02T00:00:00-05:00,9.3,Approved,,ft^3/s,https://nednr.nebraska.gov/IwipApi/api/v1/Stre...,DailyMeanByYear
0,00120500,2008-10-03T00:00:00-05:00,9.4,Approved,,ft^3/s,https://nednr.nebraska.gov/IwipApi/api/v1/Stre...,DailyMeanByYear
0,00120500,2008-10-04T00:00:00-05:00,9.1,Approved,,ft^3/s,https://nednr.nebraska.gov/IwipApi/api/v1/Stre...,DailyMeanByYear
0,00120500,2008-10-05T00:00:00-05:00,9.7,Approved,,ft^3/s,https://nednr.nebraska.gov/IwipApi/api/v1/Stre...,DailyMeanByYear


In [10]:
# Update StationNumber values must have 8 digits, with leading 0s

def updateStationNumber(x):
    x = str(x).strip()
    if len(x) == 4:
        x = "0000" + x
    if len(x) == 5:
        x = "000" + x
    if len(x) == 6:
        x = "00" + x
    if len(x) == 7:
        x = "0" + x
    return x

dfts['StationNumber'] = dfts.apply(lambda row: updateStationNumber(row['StationNumber']), axis=1)
dfts.head()

,StationNumber,Date,Value,Status,Qualifier,Unit,url,service
0,00120500,2008-10-01T00:00:00-05:00,8.8,Approved,,ft^3/s,https://nednr.nebraska.gov/IwipApi/api/v1/Stre...,DailyMeanByYear
0,00120500,2008-10-02T00:00:00-05:00,9.3,Approved,,ft^3/s,https://nednr.nebraska.gov/IwipApi/api/v1/Stre...,DailyMeanByYear
0,00120500,2008-10-03T00:00:00-05:00,9.4,Approved,,ft^3/s,https://nednr.nebraska.gov/IwipApi/api/v1/Stre...,DailyMeanByYear
0,00120500,2008-10-04T00:00:00-05:00,9.1,Approved,,ft^3/s,https://nednr.nebraska.gov/IwipApi/api/v1/Stre...,DailyMeanByYear
0,00120500,2008-10-05T00:00:00-05:00,9.7,Approved,,ft^3/s,https://nednr.nebraska.gov/IwipApi/api/v1/Stre...,DailyMeanByYear


## Output Dataframe

In [11]:
 #Merging dataframes into one, using left-join.
df = pd.merge(dfts, dfsg, on='StationNumber', how='left')
print(len(df))
df.head(1)

1392824


,StationNumber,Date,Value,Status,Qualifier,Unit,url,service,StationName,SourceName,RiverBasin,FieldOffice,StationTypeDescription,Downstream,TimeZone,Elevation,ElevationUnits,Longitude,Latitude,IsActive,WaDEUUID
0,00120500,2008-10-01T00:00:00-05:00,8.8,Approved,,ft^3/s,https://nednr.nebraska.gov/IwipApi/api/v1/Stre...,DailyMeanByYear,Parks Canal from Republican River,NeDNR,Republican River,NaN,Canal/Pump,0,MST,0.0,ft,-101.733703,40.030071,True,nebRG0


In [12]:
# output dataframe
dfout = pd.DataFrame(index=df.index)

# Site Info
dfout['in_Latitude'] = df['Latitude']
dfout['in_Longitude'] = df['Longitude']
dfout['in_PODorPOUSite'] = "Observation Site"
dfout['in_SiteName'] = df['StationName']
dfout['in_SiteNativeID'] = df['StationNumber']
dfout['in_SiteTypeCV'] = df['StationTypeDescription']

# Site VariableAmounts Info
dfout['in_Amount'] = df['Value']
dfout['in_BeneficialUseCategory'] = "Unspecified"
dfout['in_ReportYearCV'] = df['Date']
dfout['in_TimeframeEnd'] = df['Date']
dfout['in_TimeframeStart'] = df['Date']

print(len(dfout))
dfout.head(1)

1392824


,in_Latitude,in_Longitude,in_PODorPOUSite,in_SiteName,in_SiteNativeID,in_SiteTypeCV,in_Amount,in_BeneficialUseCategory,in_ReportYearCV,in_TimeframeEnd,in_TimeframeStart
0,40.030071,-101.733703,Observation Site,Parks Canal from Republican River,00120500,Canal/Pump,8.8,Unspecified,2008-10-01T00:00:00-05:00,2008-10-01T00:00:00-05:00,2008-10-01T00:00:00-05:00


## Cleaning Output

In [13]:
dfout.info()
dfout.head(1)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1392824 entries, 0 to 1392823
Data columns (total 11 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   in_Latitude               1392824 non-null  float64
 1   in_Longitude              1392824 non-null  float64
 2   in_PODorPOUSite           1392824 non-null  object 
 3   in_SiteName               1392824 non-null  object 
 4   in_SiteNativeID           1392824 non-null  object 
 5   in_SiteTypeCV             1392824 non-null  object 
 6   in_Amount                 1392824 non-null  float64
 7   in_BeneficialUseCategory  1392824 non-null  object 
 8   in_ReportYearCV           1392824 non-null  object 
 9   in_TimeframeEnd           1392824 non-null  object 
 10  in_TimeframeStart         1392824 non-null  object 
dtypes: float64(3), object(8)
memory usage: 127.5+ MB


,in_Latitude,in_Longitude,in_PODorPOUSite,in_SiteName,in_SiteNativeID,in_SiteTypeCV,in_Amount,in_BeneficialUseCategory,in_ReportYearCV,in_TimeframeEnd,in_TimeframeStart
0,40.030071,-101.733703,Observation Site,Parks Canal from Republican River,00120500,Canal/Pump,8.8,Unspecified,2008-10-01T00:00:00-05:00,2008-10-01T00:00:00-05:00,2008-10-01T00:00:00-05:00


In [14]:
# Convert History Year to YYYY-MM-DD format.
dfout['in_TimeframeEnd'] = pd.to_datetime(dfout['in_TimeframeEnd'], utc=True)
dfout['in_TimeframeEnd'] = pd.to_datetime(dfout["in_TimeframeEnd"].dt.strftime('%m/%d/%Y'))

dfout['in_TimeframeStart'] = pd.to_datetime(dfout['in_TimeframeStart'], utc=True)
dfout['in_TimeframeStart'] = pd.to_datetime(dfout["in_TimeframeStart"].dt.strftime('%m/%d/%Y'))

dfout.head(1)

,in_Latitude,in_Longitude,in_PODorPOUSite,in_SiteName,in_SiteNativeID,in_SiteTypeCV,in_Amount,in_BeneficialUseCategory,in_ReportYearCV,in_TimeframeEnd,in_TimeframeStart
0,40.030071,-101.733703,Observation Site,Parks Canal from Republican River,00120500,Canal/Pump,8.8,Unspecified,2008-10-01T00:00:00-05:00,2008-10-01,2008-10-01


In [15]:
# extract year out
dfout['in_ReportYearCV'] = pd.to_datetime(dfout['in_ReportYearCV'], utc=True)
dfout['in_ReportYearCV'] = pd.to_datetime(dfout["in_ReportYearCV"].dt.strftime('%m/%d/%Y'))
dfout['in_ReportYearCV'] = dfout['in_ReportYearCV'].dt.year
dfout.head(1)

,in_Latitude,in_Longitude,in_PODorPOUSite,in_SiteName,in_SiteNativeID,in_SiteTypeCV,in_Amount,in_BeneficialUseCategory,in_ReportYearCV,in_TimeframeEnd,in_TimeframeStart
0,40.030071,-101.733703,Observation Site,Parks Canal from Republican River,00120500,Canal/Pump,8.8,Unspecified,2008,2008-10-01,2008-10-01


In [16]:
# in_Latitude & in_Longitude
dfout['in_Latitude'] = pd.to_numeric(dfout['in_Latitude'], errors='coerce').fillna(0)
dfout['in_Longitude'] = pd.to_numeric(dfout['in_Longitude'], errors='coerce').fillna(0)
dfout.head(1)

,in_Latitude,in_Longitude,in_PODorPOUSite,in_SiteName,in_SiteNativeID,in_SiteTypeCV,in_Amount,in_BeneficialUseCategory,in_ReportYearCV,in_TimeframeEnd,in_TimeframeStart
0,40.030071,-101.733703,Observation Site,Parks Canal from Republican River,00120500,Canal/Pump,8.8,Unspecified,2008,2008-10-01,2008-10-01


## Export Outputs

In [17]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(dfout.dtypes)

in_Latitude                        float64
in_Longitude                       float64
in_PODorPOUSite                     object
in_SiteName                         object
in_SiteNativeID                     object
in_SiteTypeCV                       object
in_Amount                          float64
in_BeneficialUseCategory            object
in_ReportYearCV                      int64
in_TimeframeEnd             datetime64[ns]
in_TimeframeStart           datetime64[ns]
dtype: object


In [18]:
#Exporting to Finished File
dfout.to_csv('P_neSSRGMain.csv', index=False)  # The output